In [ ]:
import numpy as np
import pandas as pd
import tushare as ts
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from scipy import stats
from datetime import datetime, timedelta
from qvix import get_qvix, update_qvix, get_hist_vol
from option_backtest import FixedHedge, DynamicHedge


def get_iv_hv_data(underlying: str):
    qvix_df = get_qvix(underlying)
    hist_vol = get_hist_vol(underlying)
    hist_vol['HV_20_40'] = (hist_vol['HV_20'] + hist_vol['HV_40']) / 2
    hist_vol['date'] = hist_vol['date'].map(lambda d_str: datetime.strptime(d_str, '%Y-%m-%d'))
    hist_vol.set_index('date', inplace=True)
    hist_vol['hv_future'] = hist_vol['HV_20_40'].shift(-20)
    return qvix_df, hist_vol


def plot_iv_hv(qvix_df: pd.DataFrame, hv_df: pd.DataFrame, future_vol: bool = False):
    hv_list = hv_df['HV_20_40'] if not future_vol else hv_df['hv_future']
    hv_name = 'HV_20_40' if not future_vol else 'hv_future'
    
    disp_list = [qvix_df['close'], hv_list]
    df_combine = pd.concat(disp_list, axis=1).dropna()
    df_combine.head()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_combine.index, y=df_combine[hv_name], mode='lines+markers', name='HV'))
    fig.add_trace(go.Scatter(x=df_combine.index, y=df_combine['close'], mode='lines+markers', name='IV'))
    fig.show()
    
    
    
def plot_box(qvix_df: pd.DataFrame, hv_df: pd.DataFrame):
    fig = go.Figure()
    fig.add_trace(go.Box(y=qvix_df['close'], name='qvix', boxpoints='all'))
    fig.add_trace(go.Box(y=hv_df['HV_20_40'], name='hv_20&40', boxpoints='all'))
    # fig.update_traces(boxpoints='all', jitter=0)
    fig.show()
    

def plot_hv_cone(underlying: str):
    hist_vol = get_hist_vol(underlying)
    columns = [name for name in hist_vol.columns if name.startswith('HV')]
    quantile_df = hist_vol[columns].quantile(np.arange(.1, 1.1, .1))
    quantile_df.index = list(range(10, 110, 10))

    fig = go.Figure()
    for index, name in quantile_df.iterrows():
        fig.add_trace(go.Scatter(x=columns, y=quantile_df.loc[index].values, mode='lines+markers', name=str(index)))

    fig.show()

In [ ]:
# 从期权论坛更新qvix数据
_ = update_qvix('510050')
_ = update_qvix('510300')

In [ ]:
# 获取510050和510300数据
qvix_510050, hist_vol_510050 = get_iv_hv_data('510050')
qvix_510300, hist_vol_510300 = get_iv_hv_data('510300')
# qvix_510050.head()
# hist_vol_510050.head()
# qvix_510300.head()
# hist_vol_510300.head()

In [ ]:
# 绘制510050 IV和HV走势图
plot_iv_hv(qvix_510050, hist_vol_510050)

In [ ]:
# 绘制510050 IV和未来20日的（hv_20_40)
plot_iv_hv(qvix_510050, hist_vol_510050, future_vol=True)

In [ ]:
# 绘制510300 IV和HV走势图
plot_iv_hv(qvix_510300, hist_vol_510300)

In [ ]:
# 绘制510050 qvix和hv盒须图以及散点分布
plot_box(qvix_510050, hist_vol_510050)

In [ ]:
# 绘制510300 qvix和hv盒须图以及散点分布
plot_box(qvix_510300, hist_vol_510300)

In [ ]:
# 绘制510050历史波动率锥
plot_hv_cone('510050')

In [ ]:
# 绘制510300历史波动率锥
plot_hv_cone('510300')

In [ ]:
# 绘制期权论坛qvix盒须图
fig = px.box(qvix_df, y='high', points='all')
fig.show()

In [ ]:
# IV20&40盒须图
fig = px.box(hist_vol, y='HV_20_40', points='all')
fig.show()

In [ ]:
# 绘制历史波动率锥
hist_vol = get_hist_vol()
columns = [name for name in hist_vol.columns if name.startswith('HV')]
quantile_df = hist_vol[columns].quantile(np.arange(.1, 1.1, .1))
quantile_df.index = list(range(10, 110, 10))

fig = go.Figure()
for index, name in quantile_df.iterrows():
    fig.add_trace(go.Scatter(x=columns, y=quantile_df.loc[index].values, mode='lines+markers', name=str(index)))

fig.show()

In [ ]:
# 统计510050qvix节假日前一个交易日的隐波涨跌情况
df = qvix_df
df['date'] = df.index
df['last_date'] = df['date'].shift(1)
df['last_close'] = df['close'].shift(1)
df.dropna(inplace=True)
df['vix_change'] = df['close'] - df['last_close']
df['interval'] = df['date'] - df['last_date']
df['id'] = np.arange(len(df))

# 筛选数据
holiday = df[df['interval'] > timedelta(days=3)]

# 筛选节假日前一天
pre_holiday_id_list = (holiday['id'] - 1).to_list()
pre_holiday_id_list
pre_holiday = df[df['id'].isin(pre_holiday_id_list)]

print('up count:', len(pre_holiday[pre_holiday.vix_change > 0]))
print('down count:', len(pre_holiday[pre_holiday.vix_change < 0]))
print('zero count:', len(pre_holiday[pre_holiday.vix_change == 0]))
print('all count:', len(pre_holiday))

In [ ]:
# 真格量化计算出来的平值沽购隐波平均值
zg_iv = [{'date': '20190102', 'atm_iv': 0.24025449218750006}, {'date': '20190103', 'atm_iv': 0.23497841796875005}, {'date': '20190104', 'atm_iv': 0.22266757812500004}, {'date': '20190107', 'atm_iv': 0.23478300781250006}, {'date': '20190108', 'atm_iv': 0.22970234375000004}, {'date': '20190109', 'atm_iv': 0.20957509765625001}, {'date': '20190110', 'atm_iv': 0.2017603515625}, {'date': '20190111', 'atm_iv': 0.1878966796875}, {'date': '20190114', 'atm_iv': 0.21563281250000002}, {'date': '20190115', 'atm_iv': 0.19766591796875002}, {'date': '20190116', 'atm_iv': 0.18692558593750003}, {'date': '20190117', 'atm_iv': 0.1699369140625}, {'date': '20190118', 'atm_iv': 0.16759433593750003}, {'date': '20190121', 'atm_iv': 0.18653398437500002}, {'date': '20190122', 'atm_iv': 0.1738412109375}, {'date': '20190123', 'atm_iv': 0.166423046875}, {'date': '20190124', 'atm_iv': 0.15041542968750002}, {'date': '20190125', 'atm_iv': 0.14748720703125004}, {'date': '20190128', 'atm_iv': 0.16134746093750002}, {'date': '20190129', 'atm_iv': 0.17481728515625}, {'date': '20190130', 'atm_iv': 0.18184501953125}, {'date': '20190131', 'atm_iv': 0.17559814453125}, {'date': '20190201', 'atm_iv': 0.17481728515625}, {'date': '20190211', 'atm_iv': 0.17911269531250001}, {'date': '20190212', 'atm_iv': 0.17813603515624998}, {'date': '20190213', 'atm_iv': 0.195516015625}, {'date': '20190214', 'atm_iv': 0.21406953125000003}, {'date': '20190215', 'atm_iv': 0.19023925781250003}, {'date': '20190218', 'atm_iv': 0.22051806640625005}, {'date': '20190219', 'atm_iv': 0.23517382812500004}, {'date': '20190220', 'atm_iv': 0.25197910156250003}, {'date': '20190221', 'atm_iv': 0.23107021484375007}, {'date': '20190222', 'atm_iv': 0.22598955078125005}, {'date': '20190225', 'atm_iv': 0.37020224609375}, {'date': '20190226', 'atm_iv': 0.30395820312500005}, {'date': '20190227', 'atm_iv': 0.31665986328125}, {'date': '20190228', 'atm_iv': 0.29926835937500007}, {'date': '20190301', 'atm_iv': 0.29262441406250006}, {'date': '20190304', 'atm_iv': 0.3254533203125}, {'date': '20190305', 'atm_iv': 0.34264941406250005}, {'date': '20190306', 'atm_iv': 0.36101796875}, {'date': '20190307', 'atm_iv': 0.37528291015625004}, {'date': '20190308', 'atm_iv': 0.36668486328125005}, {'date': '20190311', 'atm_iv': 0.34167236328125}, {'date': '20190312', 'atm_iv': 0.34264941406250005}, {'date': '20190313', 'atm_iv': 0.33307431640625}, {'date': '20190314', 'atm_iv': 0.31333789062500006}, {'date': '20190315', 'atm_iv': 0.282072265625}, {'date': '20190318', 'atm_iv': 0.303958203125}, {'date': '20190319', 'atm_iv': 0.32252216796875005}, {'date': '20190320', 'atm_iv': 0.3162690429687501}, {'date': '20190321', 'atm_iv': 0.31939560546875007}, {'date': '20190322', 'atm_iv': 0.31079755859375}, {'date': '20190325', 'atm_iv': 0.27972734375}, {'date': '20190326', 'atm_iv': 0.26253125000000005}, {'date': '20190327', 'atm_iv': 0.25197910156250003}, {'date': '20190328', 'atm_iv': 0.24338105468750004}, {'date': '20190329', 'atm_iv': 0.26272666015625007}, {'date': '20190401', 'atm_iv': 0.29633720703125005}, {'date': '20190402', 'atm_iv': 0.30688935546875}, {'date': '20190403', 'atm_iv': 0.29965917968750005}, {'date': '20190404', 'atm_iv': 0.30200410156250007}, {'date': '20190408', 'atm_iv': 0.3160736328125}, {'date': '20190409', 'atm_iv': 0.302394921875}, {'date': '20190410', 'atm_iv': 0.3121654296875}, {'date': '20190411', 'atm_iv': 0.30669394531250005}, {'date': '20190412', 'atm_iv': 0.2723017578125001}, {'date': '20190415', 'atm_iv': 0.28089980468750003}, {'date': '20190416', 'atm_iv': 0.27503750000000005}, {'date': '20190417', 'atm_iv': 0.2754283203125001}, {'date': '20190418', 'atm_iv': 0.28109521484375}, {'date': '20190419', 'atm_iv': 0.26976142578125006}, {'date': '20190422', 'atm_iv': 0.28812998046875005}, {'date': '20190423', 'atm_iv': 0.2668302734375001}, {'date': '20190424', 'atm_iv': 0.25999091796875}, {'date': '20190425', 'atm_iv': 0.26194501953125005}, {'date': '20190426', 'atm_iv': 0.25276074218750005}, {'date': '20190429', 'atm_iv': 0.24787548828125003}, {'date': '20190430', 'atm_iv': 0.25061123046875006}, {'date': '20190506', 'atm_iv': 0.29614179687500003}, {'date': '20190507', 'atm_iv': 0.24103613281250003}, {'date': '20190508', 'atm_iv': 0.26624404296875004}, {'date': '20190509', 'atm_iv': 0.27738242187500006}, {'date': '20190510', 'atm_iv': 0.29672802734375003}, {'date': '20190513', 'atm_iv': 0.28383095703125005}, {'date': '20190514', 'atm_iv': 0.26858896484375006}, {'date': '20190515', 'atm_iv': 0.23771416015625005}, {'date': '20190516', 'atm_iv': 0.22794365234375003}, {'date': '20190517', 'atm_iv': 0.22794365234375003}, {'date': '20190520', 'atm_iv': 0.21934560546875004}, {'date': '20190521', 'atm_iv': 0.21016132812500002}, {'date': '20190522', 'atm_iv': 0.1998060546875}, {'date': '20190523', 'atm_iv': 0.208598046875}, {'date': '20190524', 'atm_iv': 0.20996591796875003}, {'date': '20190527', 'atm_iv': 0.22032265625000003}, {'date': '20190528', 'atm_iv': 0.21387695312500002}, {'date': '20190529', 'atm_iv': 0.19942187500000003}, {'date': '20190530', 'atm_iv': 0.20957519531250002}, {'date': '20190531', 'atm_iv': 0.22462167968750002}, {'date': '20190603', 'atm_iv': 0.22286298828125004}, {'date': '20190604', 'atm_iv': 0.23517382812500004}, {'date': '20190605', 'atm_iv': 0.22931152343750005}, {'date': '20190606', 'atm_iv': 0.23087480468750005}, {'date': '20190610', 'atm_iv': 0.21875937500000003}, {'date': '20190611', 'atm_iv': 0.19415166015625002}, {'date': '20190612', 'atm_iv': 0.20703486328125}, {'date': '20190613', 'atm_iv': 0.19668134765625003}, {'date': '20190614', 'atm_iv': 0.19297294921875002}, {'date': '20190617', 'atm_iv': 0.19199619140625004}, {'date': '20190618', 'atm_iv': 0.18672539062500002}, {'date': '20190619', 'atm_iv': 0.17188906250000002}, {'date': '20190620', 'atm_iv': 0.22716201171875006}, {'date': '20190621', 'atm_iv': 0.24533515625000002}, {'date': '20190624', 'atm_iv': 0.25236992187500007}, {'date': '20190625', 'atm_iv': 0.25178369140625}, {'date': '20190626', 'atm_iv': 0.25139287109375}, {'date': '20190627', 'atm_iv': 0.25940468750000006}, {'date': '20190628', 'atm_iv': 0.2736696289062501}, {'date': '20190701', 'atm_iv': 0.24982958984375003}, {'date': '20190702', 'atm_iv': 0.24201318359375007}, {'date': '20190703', 'atm_iv': 0.22794365234375003}, {'date': '20190704', 'atm_iv': 0.22833447265625}, {'date': '20190705', 'atm_iv': 0.23419677734375005}, {'date': '20190708', 'atm_iv': 0.22286298828125}, {'date': '20190709', 'atm_iv': 0.20781650390625}, {'date': '20190710', 'atm_iv': 0.2097705078125}, {'date': '20190711', 'atm_iv': 0.21406953125}, {'date': '20190712', 'atm_iv': 0.2074255859375}, {'date': '20190715', 'atm_iv': 0.217977734375}, {'date': '20190716', 'atm_iv': 0.22462167968750002}, {'date': '20190717', 'atm_iv': 0.21582822265625}, {'date': '20190718', 'atm_iv': 0.22012724609375003}, {'date': '20190719', 'atm_iv': 0.22638037109375003}, {'date': '20190722', 'atm_iv': 0.2128970703125}, {'date': '20190723', 'atm_iv': 0.19883037109375}, {'date': '20190724', 'atm_iv': 0.1894583984375}, {'date': '20190725', 'atm_iv': 0.17989287109375002}, {'date': '20190726', 'atm_iv': 0.18262587890625004}, {'date': '20190729', 'atm_iv': 0.18867822265625}, {'date': '20190730', 'atm_iv': 0.18906796875}, {'date': '20190731', 'atm_iv': 0.19219140625}, {'date': '20190801', 'atm_iv': 0.16935126953125001}, {'date': '20190802', 'atm_iv': 0.17423164062500002}, {'date': '20190805', 'atm_iv': 0.19805068359375}, {'date': '20190806', 'atm_iv': 0.21035673828125}, {'date': '20190807', 'atm_iv': 0.19708105468750003}, {'date': '20190808', 'atm_iv': 0.19181572265625002}, {'date': '20190809', 'atm_iv': 0.17989804687500002}, {'date': '20190812', 'atm_iv': 0.18107871093750003}, {'date': '20190813', 'atm_iv': 0.18458515625000002}, {'date': '20190814', 'atm_iv': 0.17110820312500002}, {'date': '20190815', 'atm_iv': 0.17540292968750001}, {'date': '20190816', 'atm_iv': 0.16759541015625}, {'date': '20190819', 'atm_iv': 0.19747714843750003}, {'date': '20190820', 'atm_iv': 0.182627734375}, {'date': '20190821', 'atm_iv': 0.17032734375}, {'date': '20190822', 'atm_iv': 0.17598857421875}, {'date': '20190823', 'atm_iv': 0.17677109375}, {'date': '20190826', 'atm_iv': 0.19044062500000003}, {'date': '20190827', 'atm_iv': 0.18184501953125004}, {'date': '20190828', 'atm_iv': 0.18204023437500003}, {'date': '20190829', 'atm_iv': 0.17950244140625005}, {'date': '20190830', 'atm_iv': 0.17715986328125}, {'date': '20190902', 'atm_iv': 0.17696474609375}, {'date': '20190903', 'atm_iv': 0.17891787109375}, {'date': '20190904', 'atm_iv': 0.16095703125000002}, {'date': '20190905', 'atm_iv': 0.17091298828125}, {'date': '20190906', 'atm_iv': 0.17540292968750001}, {'date': '20190909', 'atm_iv': 0.18926318359375}, {'date': '20190910', 'atm_iv': 0.18731103515625003}, {'date': '20190911', 'atm_iv': 0.18164980468750003}, {'date': '20190912', 'atm_iv': 0.17676943359375002}, {'date': '20190916', 'atm_iv': 0.180478515625}, {'date': '20190917', 'atm_iv': 0.16837519531250003}, {'date': '20190918', 'atm_iv': 0.16232353515625}, {'date': '20190919', 'atm_iv': 0.1644708984375}, {'date': '20190920', 'atm_iv': 0.16076181640625}, {'date': '20190923', 'atm_iv': 0.15568623046875002}, {'date': '20190924', 'atm_iv': 0.15061064453125003}, {'date': '20190925', 'atm_iv': 0.15490537109375}, {'date': '20190926', 'atm_iv': 0.16056660156250002}, {'date': '20190927', 'atm_iv': 0.16232353515625}, {'date': '20190930', 'atm_iv': 0.16427568359375003}, {'date': '20191008', 'atm_iv': 0.14943935546875}, {'date': '20191009', 'atm_iv': 0.13733603515625004}, {'date': '20191010', 'atm_iv': 0.1418259765625}, {'date': '20191011', 'atm_iv': 0.13538388671875}, {'date': '20191014', 'atm_iv': 0.155491015625}, {'date': '20191015', 'atm_iv': 0.14631591796875001}, {'date': '20191016', 'atm_iv': 0.14729199218750003}, {'date': '20191017', 'atm_iv': 0.14846328125000002}, {'date': '20191018', 'atm_iv': 0.1394833984375}, {'date': '20191021', 'atm_iv': 0.1344078125}, {'date': '20191022', 'atm_iv': 0.12874658203125003}, {'date': '20191023', 'atm_iv': 0.13226044921875002}, {'date': '20191024', 'atm_iv': 0.132065234375}, {'date': '20191025', 'atm_iv': 0.12659921875}, {'date': '20191028', 'atm_iv': 0.1269896484375}, {'date': '20191029', 'atm_iv': 0.130503515625}, {'date': '20191030', 'atm_iv': 0.13362695312500003}, {'date': '20191031', 'atm_iv': 0.13401738281250003}, {'date': '20191101', 'atm_iv': 0.13187001953125005}, {'date': '20191104', 'atm_iv': 0.13245566406250003}, {'date': '20191105', 'atm_iv': 0.13401738281250003}, {'date': '20191106', 'atm_iv': 0.13147958984375002}, {'date': '20191107', 'atm_iv': 0.13616474609375}, {'date': '20191108', 'atm_iv': 0.13440781250000003}, {'date': '20191111', 'atm_iv': 0.13499345703125004}, {'date': '20191112', 'atm_iv': 0.12718486328125003}, {'date': '20191113', 'atm_iv': 0.12913701171875}, {'date': '20191114', 'atm_iv': 0.13304130859375}, {'date': '20191115', 'atm_iv': 0.12484228515625001}, {'date': '20191118', 'atm_iv': 0.12289013671875001}, {'date': '20191119', 'atm_iv': 0.114886328125}, {'date': '20191120', 'atm_iv': 0.10473515625000002}, {'date': '20191121', 'atm_iv': 0.12425664062500003}, {'date': '20191122', 'atm_iv': 0.12816093750000002}, {'date': '20191125', 'atm_iv': 0.12581835937500002}, {'date': '20191126', 'atm_iv': 0.12913701171875003}, {'date': '20191127', 'atm_iv': 0.1254279296875}, {'date': '20191128', 'atm_iv': 0.12289013671875001}, {'date': '20191129', 'atm_iv': 0.12679443359375003}, {'date': '20191202', 'atm_iv': 0.1246470703125}, {'date': '20191203', 'atm_iv': 0.12484228515625001}, {'date': '20191204', 'atm_iv': 0.12484228515625001}, {'date': '20191205', 'atm_iv': 0.11957148437500002}, {'date': '20191206', 'atm_iv': 0.11156767578125001}, {'date': '20191209', 'atm_iv': 0.11664326171875002}, {'date': '20191210', 'atm_iv': 0.11508154296875}, {'date': '20191211', 'atm_iv': 0.11078681640625002}, {'date': '20191212', 'atm_iv': 0.11410546875000001}, {'date': '20191213', 'atm_iv': 0.12464707031250002}, {'date': '20191216', 'atm_iv': 0.10668730468750001}, {'date': '20191217', 'atm_iv': 0.11664326171875002}, {'date': '20191218', 'atm_iv': 0.12738007812500002}, {'date': '20191219', 'atm_iv': 0.14397333984375002}, {'date': '20191220', 'atm_iv': 0.13928818359375}, {'date': '20191223', 'atm_iv': 0.13499345703125}, {'date': '20191224', 'atm_iv': 0.12289013671875001}, {'date': '20191225', 'atm_iv': 0.12464707031250002}, {'date': '20191226', 'atm_iv': 0.12406142578125001}, {'date': '20191227', 'atm_iv': 0.12659921875000002}, {'date': '20191230', 'atm_iv': 0.14241162109375002}, {'date': '20191231', 'atm_iv': 0.14416855468750003}, {'date': '20200102', 'atm_iv': 0.14573027343750003}, {'date': '20200103', 'atm_iv': 0.14084990234375003}, {'date': '20200106', 'atm_iv': 0.14475419921875002}, {'date': '20200107', 'atm_iv': 0.1574431640625}, {'date': '20200108', 'atm_iv': 0.1551005859375}, {'date': '20200109', 'atm_iv': 0.14846328125000002}, {'date': '20200110', 'atm_iv': 0.14299726562500004}, {'date': '20200113', 'atm_iv': 0.13987382812500002}, {'date': '20200114', 'atm_iv': 0.13870253906250002}, {'date': '20200115', 'atm_iv': 0.13987382812500002}, {'date': '20200116', 'atm_iv': 0.1308939453125}, {'date': '20200117', 'atm_iv': 0.12777050781250002}, {'date': '20200120', 'atm_iv': 0.13714082031250002}, {'date': '20200121', 'atm_iv': 0.15158671875000002}, {'date': '20200122', 'atm_iv': 0.13792167968750002}, {'date': '20200123', 'atm_iv': 0.158614453125}, {'date': '20200203', 'atm_iv': 0.3521023803710937}, {'date': '20200204', 'atm_iv': 0.21485830078125}, {'date': '20200205', 'atm_iv': 0.19044687500000002}, {'date': '20200206', 'atm_iv': 0.18809746093750002}, {'date': '20200207', 'atm_iv': 0.20704072265625004}, {'date': '20200210', 'atm_iv': 0.20236083984375006}, {'date': '20200211', 'atm_iv': 0.17560058593750003}, {'date': '20200212', 'atm_iv': 0.16798476562500003}, {'date': '20200213', 'atm_iv': 0.16993691406250003}, {'date': '20200214', 'atm_iv': 0.15646708984375002}, {'date': '20200217', 'atm_iv': 0.16115224609375003}, {'date': '20200218', 'atm_iv': 0.15822402343750003}, {'date': '20200219', 'atm_iv': 0.15334365234375003}, {'date': '20200220', 'atm_iv': 0.17540292968750001}, {'date': '20200221', 'atm_iv': 0.18633496093750002}, {'date': '20200224', 'atm_iv': 0.20177167968750004}, {'date': '20200225', 'atm_iv': 0.21582822265625}, {'date': '20200226', 'atm_iv': 0.2261849609375}, {'date': '20200227', 'atm_iv': 0.18926318359375}, {'date': '20200228', 'atm_iv': 0.23224267578125002}, {'date': '20200302', 'atm_iv': 0.1978583984375}, {'date': '20200303', 'atm_iv': 0.19277705078125}, {'date': '20200304', 'atm_iv': 0.19512070312500002}, {'date': '20200305', 'atm_iv': 0.19629453125000002}, {'date': '20200306', 'atm_iv': 0.21739150390625}, {'date': '20200309', 'atm_iv': 0.286762109375}, {'date': '20200310', 'atm_iv': 0.2840263671875}, {'date': '20200311', 'atm_iv': 0.28598046875000005}, {'date': '20200312', 'atm_iv': 0.31118837890625006}, {'date': '20200313', 'atm_iv': 0.41133608398437504}, {'date': '20200316', 'atm_iv': 0.45955354003906246}, {'date': '20200317', 'atm_iv': 0.4343456298828125}, {'date': '20200318', 'atm_iv': 0.3656833862304688}, {'date': '20200319', 'atm_iv': 0.36277666015625}, {'date': '20200320', 'atm_iv': 0.31392412109375006}, {'date': '20200323', 'atm_iv': 0.36707568359375004}, {'date': '20200324', 'atm_iv': 0.33541923828125003}, {'date': '20200325', 'atm_iv': 0.3254533203125}, {'date': '20200326', 'atm_iv': 0.32682119140625004}, {'date': '20200327', 'atm_iv': 0.33444218750000004}, {'date': '20200330', 'atm_iv': 0.32623496093750004}, {'date': '20200331', 'atm_iv': 0.3147057617187501}, {'date': '20200401', 'atm_iv': 0.2871529296875}, {'date': '20200402', 'atm_iv': 0.26702568359375}, {'date': '20200403', 'atm_iv': 0.27992275390625004}, {'date': '20200407', 'atm_iv': 0.2666348632812501}, {'date': '20200408', 'atm_iv': 0.24748466796875004}, {'date': '20200409', 'atm_iv': 0.23439218750000004}, {'date': '20200410', 'atm_iv': 0.21836962890625006}, {'date': '20200413', 'atm_iv': 0.20938623046875002}, {'date': '20200414', 'atm_iv': 0.19005644531250002}, {'date': '20200415', 'atm_iv': 0.19434609375}, {'date': '20200416', 'atm_iv': 0.19494169921875001}, {'date': '20200417', 'atm_iv': 0.19592382812500003}, {'date': '20200420', 'atm_iv': 0.19573134765625005}, {'date': '20200421', 'atm_iv': 0.19026689453125004}, {'date': '20200422', 'atm_iv': 0.19339052734375003}, {'date': '20200423', 'atm_iv': 0.18987470703125}, {'date': '20200424', 'atm_iv': 0.17697333984375002}, {'date': '20200427', 'atm_iv': 0.1728724609375}, {'date': '20200428', 'atm_iv': 0.16740703125}, {'date': '20200429', 'atm_iv': 0.18186113281250002}, {'date': '20200430', 'atm_iv': 0.19709121093750004}, {'date': '20200506', 'atm_iv': 0.18927675781250003}, {'date': '20200507', 'atm_iv': 0.17658212890625002}, {'date': '20200508', 'atm_iv': 0.156271875}, {'date': '20200511', 'atm_iv': 0.16369003906250001}, {'date': '20200512', 'atm_iv': 0.16329960937500002}, {'date': '20200513', 'atm_iv': 0.15861445312500003}, {'date': '20200514', 'atm_iv': 0.1675943359375}, {'date': '20200515', 'atm_iv': 0.15431972656250004}, {'date': '20200518', 'atm_iv': 0.15256279296875003}, {'date': '20200519', 'atm_iv': 0.14202119140625}, {'date': '20200520', 'atm_iv': 0.14690156250000003}, {'date': '20200521', 'atm_iv': 0.16525175781250004}, {'date': '20200522', 'atm_iv': 0.19064501953125002}, {'date': '20200525', 'atm_iv': 0.1652517578125}, {'date': '20200526', 'atm_iv': 0.16134746093750002}, {'date': '20200527', 'atm_iv': 0.1713064453125}, {'date': '20200528', 'atm_iv': 0.17638574218750003}, {'date': '20200529', 'atm_iv': 0.1802953125}, {'date': '20200601', 'atm_iv': 0.15490537109375002}, {'date': '20200602', 'atm_iv': 0.16115390625}, {'date': '20200603', 'atm_iv': 0.16603330078125}, {'date': '20200604', 'atm_iv': 0.168961328125}, {'date': '20200605', 'atm_iv': 0.15685751953125}, {'date': '20200608', 'atm_iv': 0.15880966796875}, {'date': '20200609', 'atm_iv': 0.15373408203125002}, {'date': '20200610', 'atm_iv': 0.165838671875}, {'date': '20200611', 'atm_iv': 0.17482968750000003}, {'date': '20200612', 'atm_iv': 0.15979169921875}, {'date': '20200615', 'atm_iv': 0.18304296875000003}]

In [ ]:
zg_df = pd.DataFrame(zg_iv)
zg_df['date'] = zg_df['date'].map(lambda date: datetime.strptime(date, '%Y%m%d'))
zg_df['atm_iv'] = zg_df['atm_iv'] * 100
zg_df.set_index('date', inplace=True)

In [ ]:
disp_list = [qvix_510050['close'], zg_df['atm_iv']]
df_combine = pd.concat(disp_list, axis=1).dropna()
df_combine.head()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_combine.index, y=df_combine['atm_iv'], mode='lines+markers', name='zg_atm_iv'))
fig.add_trace(go.Scatter(x=df_combine.index, y=df_combine['close'], mode='lines+markers', name='qvix'))
fig.show()